## **Жадный алгоритм множественного выравнивания** (7 баллов)
Реализуйте алгоритм, который принимал бы на вход
массив строк, штраф за удаления, вставки и несовпадения,
а так-же цену совпадений. А возвращал бы множественное
выравнивание. На первом шаге алгоритм должен выбрать
две самые близкие по расстоянию Левенштейна строки и
заменить их консенснусной строкой. При следующих шагах
алгоритма выравниваться между собой могут так же и
консенснусные строки. При этом стоит хранить для каждой
строки не только ее саму но и профиль множественного
выравнивания, чтобы в итоге правильно пересчитывать
консенсус.  
Результат работы алгоритма - массив строк,
соответствующий некоторому множественному выравниванию.


In [146]:
import numpy as np

class Msa:

    def __init__(self, seqs):
        self.seqs = seqs
        self.lenght = len(seqs[0])
        self.calc_profile()
        if len(seqs) > 1:
            self.calc_consensus()
        else:
            self.consensus = seqs[0]

    def calc_profile(self):
        seqs = np.array([list(x) for x in self.seqs]).T
        #fprint(seqs)
        self.msa_profile = [{nuc: np.count_nonzero(seqs[i] == nuc) for nuc in 'AGTC-'} for i in range(self.lenght)]

    def calc_consensus(self):
        self.consensus = ''.join([max(nucs_in_pos, key=nucs_in_pos.get) for nucs_in_pos in self.msa_profile])

    def update(self, aligned_msa):
        tmp = self.consensus + '*' * len(aligned_msa)
        for num, seq in enumerate(self.seqs):
            new_seq = []
            i = 0
            for x in aligned_msa:
                if x == '-' and tmp[i] != '-':
                    new_seq.append('-')
                else:
                    new_seq.append(seq[i])
                    i += 1
            self.seqs[num] = ''.join(new_seq)

    def __repr__(self):
        return f'Seqs: {self.seqs} & Consensus: {self.consensus}'

class Msa_creator:

    def __init__(self, seqs, delet=-1, ins=-1, match=1, mismatch=-1):
        self.msas = [Msa([seq]) for seq in seqs]
        self.delet = delet
        self.ins = ins
        self.match = match
        self.mismatch = mismatch

    def align_pair(self, a, b, return_alignment=False):

        n, m = len(a) + 1, len(b) + 1

        d = [[0] * m for _ in range(n)]
        for i in range(n):
            d[i][0] = i * self.ins
        for j in range(m):
            d[0][j] = j * self.delet
        for i in range(1, n):
            for j in range(1, m):
                if a[i - 1] != b[j - 1]:
                    is_match = self.mismatch
                else:
                    is_match = self.match
                d[i][j] = max(d[i - 1][j] + self.ins, d[i][j - 1] + self.delet,
                                  d[i - 1][j - 1] + is_match)

        if return_alignment:

            i, j = n - 1, m - 1
            seq_a = []
            seq_b = []
            while i > 0 or j > 0:
                score = d[i][j]
                score_up = d[i][j - 1]
                score_left = d[i - 1][j]

                if score == score_left + self.ins and i > 0:
                    seq_a.append(a[i - 1])
                    seq_b.append('-')
                    i -= 1
                elif score == score_up + self.delet and j > 0:
                    seq_a.append('-')
                    seq_b.append(b[j - 1])
                    j -= 1

                else:
                    seq_a.append(a[i - 1])
                    seq_b.append(b[j - 1])
                    i -= 1
                    j -= 1
            seq_a.reverse()
            seq_b.reverse()
            return (d[-1][-1], [''.join(seq_a), ''.join(seq_b)])

        return d[-1][-1]

    def make_msa(self):
        n = len(self.msas)
        distance_matrix = np.eye(n)
        np.fill_diagonal(distance_matrix, -float('inf'))

        def fill_dist_matrix():
            for i in range(n):
              for j in range(i + 1, n):
                  if distance_matrix[i][j] != -float('inf'):
                      distance_matrix[i][j] = self.align_pair(self.msas[i].consensus, self.msas[j].consensus)
                      distance_matrix[j][i] = distance_matrix[i][j]

        for _ in range(n - 1):
            fill_dist_matrix()
            i, j = np.unravel_index(np.argmax(distance_matrix, axis=None), distance_matrix.shape)
            _, aligned_pair = self.align_pair(self.msas[i].consensus, self.msas[j].consensus, return_alignment=True)
            self.msas[i].update(aligned_pair[0])
            self.msas[j].update(aligned_pair[1])
            new_msa_seqs = self.msas[i].seqs
            new_msa_seqs.extend(self.msas[j].seqs)

            self.msas[i] = Msa(new_msa_seqs)
            distance_matrix[:, j] = -float('inf')
            distance_matrix[j, :] = -float('inf')


        return self.msas[i].seqs, self.msas[i].consensus





In [160]:
test_num = 2
seqs_1 = ["ACT", "ATC", "GCT", "ATCC"]
seqs_2 = ['TCGGGGTTTTT', 'CCTGACTTAC', 'ACGGGATTTTC', 'AGTC', 'TTGGGGACTTCC', 'TCGGATTCAT', 'GGGATTCC', 'TAGGGGAACC', 'TCGGGTATAACC']

cost_del=-1
cost_ins=-1
cost_match=1
cost_mismatch=-1
s = [f'seqs_{i}' for i in range(1, test_num + 1)]

for i, seqs_name in enumerate(s):
    print(f'__test__{i + 1}')
    seqs = globals()[seqs_name]
    msa, cons = Msa_creator(seqs, cost_del, cost_ins, cost_match, cost_mismatch).make_msa()
    print('Sequences:', *msa, 'Consensus:', cons, sep = '\n')
    print()



__test__1
Sequences:
A-CT
G-CT
ATC-
ATCC
Consensus:
ATCT

__test__2
Sequences:
TCGGG-GTTTTT-
ACGGG-ATTTTC-
TCGG--ATTCAT-
TTGGGGA-CTTCC
--GGG-A--TTCC
TAGGGGA-A--CC
TCGGGTATAA-CC
CCTG--ACTTAC-
A-G----T---C-
Consensus:
TCGGG-ATTTTC-

